In [1]:
import pandas as pd

In [2]:
#ref) https://medium.com/@ageitgey/quick-tip-the-easiest-way-to-grab-data-out-of-a-web-page-in-python-7153cecfca58
def download_table(url): 
    dfs = pd.read_html(url, header=0) # read tables and save them into a list of dataframes
    df = dfs[0]  # select the first element of the list
    df.dropna(axis=0, inplace=True) # axis=0: remove rows that contain NaN
    ii = df.iloc[:,0].str.contains('\d').values # excluding non-digit rows by checking the first column
    return df.loc[ii,]
def download_tables(url_, start=1, end=5, print_url = False): # read table data from the start page to end page
    dfs = list()
    for i in range(start, end+1):
        url = url_ + str(i) # create a complete url for naver web page
        if (print_url == True):
            print(url)
        df0 = download_table(url) # download a table from a naver web page
        if (i != start) and (pd.DataFrame.equals(dfs[-1], df0) or len(df0.index)==0): # stop conditions
            break
        else:
            dfs.append(df0)
    df = pd.concat(dfs)
    return df

In [3]:
def update_data_in_db(filepath, url_, COL_DATE = '날짜', MAX_PAGE = 99999, print_url = False):
    try:
        df = pd.read_csv(filepath) # read the already sorted data
        latestDate = df.iloc[-1][COL_DATE] # temporarily save the latest date
        for i in range(1, MAX_PAGE):
            url = url_ + str(i) # create a complete url for naver web page
            df0 = download_table(url)
            if any(df0[COL_DATE] == latestDate):
                df.drop(df.index[df[COL_DATE] == latestDate], inplace=True) # drop the last row 
                df = pd.concat([df, df0[df0[COL_DATE] >= latestDate]])
                break
            else:
                df = pd.concat([df, df0])
    except:
        print("File not found in the database. Now, searching the internet to download information...")
        df = download_tables(url_, start = 1, end = MAX_PAGE, print_url = print_url) # download the data from the website
        print("Downloading completed.")
    df.sort_values(COL_DATE, inplace = True) # sort in order of date & time
    df.to_csv(filepath, index=False) # save the data to db
    return df

In [6]:
if False:
#     df = download_table('http://finance.naver.com/item/sise_day.nhn?code=035420&page=1')
#     df = download_table('http://info.finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_USDKRW&page=1')
    df = download_table('https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_CADKRW&page=403')
    print(df)
if False:
#     df = download_tables('http://finance.naver.com/item/sise_day.nhn?code=035420&page=', 1, 15)
#     df = download_tables('https://finance.naver.com/item/sise_day.nhn?code=000660&page=', 542, 547, print_url = True)
    df = download_tables('https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_CADKRW&page=', 376, 380, print_url = True)
    print(df)
if False:
#    df = update_data_in_db('c:/Users/bomso/bomsoo1/project/python/naver.csv', 'http://finance.naver.com/item/sise_day.nhn?code=035420&page=')
#    df = update_data_in_db('c:/Users/bomso/bomsoo1/project/python/samsung.csv', 'http://info.finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_USDKRW&page=')
#    df = update_data_in_db('c:/Users/bomso/bomsoo1/project/python/kospi.csv', 'http://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page=')
#     df = update_data_in_db('c:/Users/bomso/bomsoo1/project/python/gold.csv', 'http://info.finance.naver.com/marketindex/goldDailyQuote.nhn?&page=')
    df = update_data_in_db('C:/Users/bomsoo/bomsoo1/project/python/project_stock_db_csv/CADKRW.csv', 'https://finance.naver.com/marketindex/exchangeDailyQuote.nhn?marketindexCd=FX_CADKRW&page=')
    
    #df['종가'].plot(xlim=(pd.Timestamp('2016-01-01'), pd.Timestamp.today()), ylim=(400000,1000000))

    print(df.iloc[-35:])

              날짜   매매기준율  전일대비      현찰    현찰.1      송금    송금.1 T/C 사실때  \
3728  2019.04.11  854.05  0.23  870.87  837.23  862.59  845.51  866.86   
3729  2019.04.12  852.96  1.09  869.76  836.16  861.48  844.44  865.75   
3730  2019.04.15  850.34  2.62  867.09  833.59  858.84  841.84  863.09   
3731  2019.04.16  849.02  1.32  865.74   832.3  857.51  840.53  861.75   
3732  2019.04.17  850.85  1.83  867.61  834.09  859.35  842.35  863.61   
3733  2019.04.18   851.1  0.25  867.86  834.34  859.61  842.59  863.86   
3734  2019.04.19  850.16  0.94   866.9  833.42  858.66  841.66  862.91   
3735  2019.04.22  854.86   4.7   871.7  838.02   863.4  846.32  867.68   
3736  2019.04.23  854.43  0.43  871.26   837.6  862.97  845.89  867.24   
3737  2019.04.24  855.76  1.33  872.61  838.91  864.31  847.21  868.59   
3738  2019.04.25  860.75  4.99   877.7   843.8  869.35  852.15  873.66   
3739  2019.04.26  860.54  0.21  877.49  843.59  869.14  851.94  873.44   
3740  2019.04.29  861.45  0.91  878.42